In [ ]:
import numpy as np

In [ ]:
def make_synthetic_dataset(n: int = 10000, rng_seed: int = 12435):
    rng = np.random.RandomState(rng_seed)
    
    x = rng.uniform(low=-1.0, high=+1.0, size=(n, 3))
    
    # Simple function for the mean of the target
    y_zero_noise = x[:, 0] - x[:, 1]
    
    # Also simple function for the stddev. Ranges from 1 to 3
    sigma = 2.0 + x[:, 2]
    eps = sigma * rng.normal(loc=0.0, scale=1.0)
    
    y_with_noise = y_zero_noise + eps
    
    return x, y_zero_noise

In [ ]:
import torch
import torch.nn as nn

In [ ]:
mu_model = nn.Linear(in_features=2, out_features=1, bias=True)
sigma_model = nn.Linear(in_features=2, out_features=1, bias=True)

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
x_data, y_data = make_synthetic_dataset()
dataset = TensorDataset(torch.Tensor(x_data), torch.Tensor(y_data))